In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2
import mediapipe as mp
import torch
import pandas as pd
from scipy.fft import fft, ifft


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
torch.cuda.empty_cache()

Using cuda device


In [3]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image



def getFiturLandmark(img):
    err = None
    detection_result = []
    try:
        
        base_options = python.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
        options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        output_segmentation_masks=True)
        detector = vision.PoseLandmarker.create_from_options(options)
        
        detection_result = detector.detect(img)

        err = None
    
    except:
        err = 1
        detection_result = []
    
    return detection_result, err



    
    # concate 
def flatten(data):
    a = np.ravel(data)
    return a


In [4]:
from torchvision import transforms
train_transform = transforms.Compose([
    transforms.ToPILImage(),
     transforms.RandomRotation(degrees=(30, 70)),
    transforms.Resize((100, 100)),
    # transforms.Grayscale(1),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
   
    transforms.ToTensor(),
   
])

In [5]:
import os

dictClass = {}
i = 0
for _i in os.listdir('../../dataset/extractFrame'):
    # print(_i)
    dictClass[i] = _i
    i = i + 1

print(dictClass)
print(dictClass[0])
for i in dictClass:
    print(dictClass[i])
    
_idClass = [_key for _key in dictClass if dictClass[_key] == "wolf"]
print(int(_idClass[0]))


{0: 'above', 1: 'accomplish', 2: 'adopt', 3: 'advantage', 4: 'alphabet', 5: 'anniversary', 6: 'another', 7: 'appropriate', 8: 'arrogant', 9: 'artist', 10: 'background', 11: 'bee', 12: 'behavior', 13: 'bell', 14: 'birth', 15: 'bless', 16: 'blood', 17: 'boots', 18: 'brave', 19: 'breathe', 20: 'bright', 21: 'bull', 22: 'butterfly', 23: 'card', 24: 'category', 25: 'catholic', 26: 'cent', 27: 'chemistry', 28: 'cherry', 29: 'china', 30: 'chop', 31: 'christian', 32: 'cigarette', 33: 'clever', 34: 'clown', 35: 'coach', 36: 'coat', 37: 'cochlear implant', 38: 'coconut', 39: 'common', 40: 'commute', 41: 'control', 42: 'count', 43: 'culture', 44: 'daily', 45: 'defend', 46: 'degree', 47: 'demonstrate', 48: 'department', 49: 'die', 50: 'dig', 51: 'dinner', 52: 'dinosaur', 53: 'diploma', 54: 'director', 55: 'disconnect', 56: 'discover', 57: 'don_t want', 58: 'drum', 59: 'pervideo', 60: 'sun', 61: 'support', 62: 'suppose', 63: 'sure', 64: 'surprise', 65: 'sweater', 66: 'swim', 67: 'symbol', 68: 'tast

In [6]:
# create dataset class
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.cluster import KMeans
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler

standardScaler = StandardScaler()

def toTensor(_x):
    # _em = torch.rand(168,51)
    # _newX = torch.as_tensor(_x)
    # _padSeq = pad_sequence([_em, _newX], batch_first=True, padding_value=0)
    # _xDt = pd.Series(_x)
    return _x



class dataClass(Dataset):
    def __init__(self, pickle_data, root_dir, transform=None):
        self.file = pd.read_pickle(pickle_data)
        self.input = torch.tensor
        self.root_dir = root_dir
        self.transform = transform
        # self.dataFitur = [np.array(_x) ]
        # self.dataScaled = standardScaler()
        
        self.yoloTensor = [torch.tensor(np.array(standardScaler.fit_transform(_x))) for _x in self.file['YOLO11']]
        self.yoloTensorPad = pad_sequence(self.yoloTensor, batch_first=True)
        
        # self.yoloTensorPackPad = pack_padded_sequence(self.yoloTensorPad, lengths=self.leng, batch_first=True, enforce_sorted=False)
        
    
    def __len__(self):
        return len(self.yoloTensorPad)
    
    def __getitem__(self, index):
        _class = self.file.iloc[index]['class']
        _image = self.file.iloc[index]['image']
        _fitur = self.yoloTensorPad[index].float()
        _idClass = [_key for _key in dictClass if dictClass[_key] == _class]
        
        _datareturn = {
                    'dataPack': _fitur,
                    'target': _class,
                    'idClass': [_idClass[0]],
                    # 'image': _image
                       }
        
            
        return _datareturn

data_torch = dataClass(pickle_data="../../dataset/pickle_YOLO", root_dir="../../dataset/extractFrame", transform=None)


In [7]:
# aaa = pd.read_pickle("../../dataset/pickle_YOLO")


In [8]:
# scaled = standardScaler.fit_transform(aaa['YOLO11'][0])
# print(scaled)
# print(aaa['YOLO11'][0])

In [9]:
print(data_torch[90]['dataPack'])
print(data_torch[90]['target'])
print(data_torch[90]['idClass'])

tensor([[ 2.6293,  0.6821, -0.7355,  ...,  0.0000,  0.0000, -0.2833],
        [ 2.2485,  0.5858, -0.7390,  ...,  0.0000,  0.0000, -0.2673],
        [ 2.1693,  0.5470, -0.6406,  ...,  0.0000,  0.0000, -0.2821],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
boots
[17]


In [10]:
from torch.utils.data import DataLoader, random_split

# split dataset
print(len(data_torch))
train_data, val_data = random_split(data_torch, [0.80, 0.20])


train_dataloader = DataLoader(train_data, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=True)

    


551


In [11]:
# model
import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 

class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True, bidirectional=False) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
    
    def forward(self,x):
        hn = 0
        h_0 = Variable(torch.zeros(self.num_layers, self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, self.hidden_size)) #internal state
        _leng = [len(_i) for _i in x]
        _packpad = pack_padded_sequence(x, lengths=_leng, batch_first=True, enforce_sorted=False)
        # print(x.shape)
        # print(_packpad)
        
        output, (hn, cn) = self.lstm(_packpad) #lstm with input, hidden, and internal state
        
        
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer nhidd
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        # out = self.softmax(out) #relu
        out = self.fc(out) #Final Output
        # out = self.softmax(out)
        return out

In [12]:
import torch.optim as optim
# train
num_epochs = 1000 #1000 epochs
learning_rate = 0.1 #0.001 lr

input_size = 51 #number of features
hidden_size = 20 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 100 #number of output classes 


lstm1 = LSTM1(num_classes=num_classes, hidden_size=hidden_size,
              input_size=input_size, num_layers=num_layers, seq_length=128) #our lstm class sss

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm1.parameters(), lr=learning_rate)


In [14]:
# train
# _leng = [len(_i) for _i in x]
# _packpad = pack_padded_sequence(x, lengths=_leng, batch_first=True, enforce_sorted=False)

current_loss = 0
all_losses = []
for epoch in range(num_epochs):
   
    for _i, _v in enumerate(train_dataloader):
        total_correct = 0
        total_samples = 0
        batch_loss = 0
        # print(_v['dataPack'].shape)
        for _in in range (0, train_dataloader.batch_size-1):
            
            # print(_v['dataPack'][_in])
        
           
            _fitur = _v['dataPack'][_in]
            # rubah dimensi menjadi (layer, seq length, fitur)
            _fitur = torch.permute(_fitur, (1,0,2)).size()
            _class = torch.tensor(_v['idClass'][_in])
            print(_class)
            # print(_class.shape)
            outputs = lstm1(_fitur)
            # print(outputs)
            
            loss = criterion(outputs, _class)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == _class).sum().item()
        
            total_samples += _class
            
            batch_loss += loss / len(_v)
            correct = (predicted == _class).sum().item() 
            loss.backward()
            optimizer.step()
        
            optimizer.zero_grad()
    # current_loss += batch_loss.item() / len(_v)
    # validation_accuracy = total_correct / total_samples

    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {batch_loss}')
    # print(f'Validation Accuracy: {validation_accuracy:.4f}')
    # all_losses.append(current_loss / len(_batch) )
    # current_loss = 0
        

KeyboardInterrupt: 